In [42]:
# pip uninstall plotly
# conda uninstall plotly
# conda install -c plotly plotly

In [1]:
import plotly.express as px
import plotly.graph_objects as go

In [5]:
# 基本ライブラリ
import pandas as pd
import pandas.io.sql as psql
import numpy as np
import numpy.random as rd
import gc
import multiprocessing as mp
import os
import sys
from collections import defaultdict
from glob import glob
import math
from datetime import datetime as dt
from pathlib import Path
import scipy.stats as st
import re
import shutil
from tqdm import tqdm_notebook as tqdm
import datetime
ts_conv = np.vectorize(datetime.datetime.fromtimestamp) # 秒ut(10桁) ⇒ 日付

import pickle
def unpickle(filename):
    with open(filename, 'rb') as fo:
        p = pickle.load(fo)
    return p

def to_pickle(filename, obj):
    with open(filename, 'wb') as f:
        pickle.dump(obj, f, -1)

# pandas settings
pd.set_option("display.max_colwidth", 100)
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.options.display.float_format = '{:,.5f}'.format

# グラフ描画系
import matplotlib
from matplotlib import font_manager
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib import rc
from matplotlib_venn import venn2, venn2_circles
from matplotlib import animation as ani
from IPython.display import Image

plt.rcParams["patch.force_edgecolor"] = True
#rc('text', usetex=True)
from IPython.display import display # Allows the use of display() for DataFrames
import seaborn as sns
sns.set(style="whitegrid", palette="muted", color_codes=True)
sns.set_style("whitegrid", {'grid.linestyle': '--'})
red = sns.xkcd_rgb["light red"]
green = sns.xkcd_rgb["medium green"]
blue = sns.xkcd_rgb["denim blue"]

%matplotlib inline
%config InlineBackend.figure_format='retina'

In [37]:
np.random.seed(71)
n = 300
x = pd.DataFrame(st.multivariate_normal(mean=[0, 0, 0], cov=1).rvs(size=n), 
                 columns=["x1", "x2", "x3"])
x["type"] = np.random.randint(5, size=n)
x["size"] = np.random.randint(10, size=n)

In [38]:
x.head()

,x1,x2,x3,type,size
0,-0.43060,-1.19393,-0.44430,1,9
1,0.48941,-0.45156,0.58570,3,1
2,1.17732,-0.96501,0.21828,1,2
3,-0.86614,-0.32301,1.41292,3,7
4,-0.71265,-1.36219,-1.70597,1,1


In [47]:
px.scatter(x, x="x1", y="x2", size="size", color="type",
                 width=1000, height=1000)

In [46]:
px.scatter_3d(x, x="x1", y="x2", z="x3", size="size", color="type",
                 width=1000, height=1000)

In [52]:
x["x1_cumsum"] = x.x1.cumsum()
x["x2_cumsum"] = x.x2.cumsum()
x["x3_cumsum"] = x.x3.cumsum()

In [62]:
x_ = pd.DataFrame(x[["x1_cumsum", "x2_cumsum", "x3_cumsum"]].stack()).reset_index()
x_.columns = ["type", "name", "val"]

In [66]:
px.line(x_, y="val", color="name",)

In [67]:
px.area(x_, y="val", color="name",)

In [70]:
px.histogram(x, x="x1", marginal="rug", nbins=30)

In [90]:
import plotly.graph_objs as go
import ipywidgets as widgets


def rebinnable_interactive_histogram(series, initial_bin_width=1., min=0.1, max=5, step=.1):
    figure_widget = go.FigureWidget(
        data=[go.Histogram(x=series, xbins={"size": initial_bin_width})]
    )

    bin_slider = widgets.FloatSlider(
        value=initial_bin_width,
        min=min,
        max=max,
        step=step,
        description="Bin width:",
        readout_format=".0f",  # display as integer
    )

    histogram_object = figure_widget.data[0]

    def set_bin_size(change):
        histogram_object.xbins = {"size": change["new"]}

    bin_slider.observe(set_bin_size, names="value")

    output_widget = widgets.VBox([figure_widget, bin_slider])
    return output_widget



In [92]:
rebinnable_interactive_histogram(x.x1)

    'data': [{'type': 'histogram',
              'uid': '8533c540-4715-49aa-aa56…

In [93]:
px.box(x_, y="val", color="name",)

In [94]:
px.violin(x_, y="val", color="name",)

In [96]:
px.density_heatmap(x, x="x1", y="x2", color_continuous_scale=px.colors.sequential.Plasma, 
                   marginal_x="histogram", marginal_y="histogram", width=1000, height=1000)


In [111]:
np.random.seed(71)
x_list = []
for year in np.arange(2010, 2021):
    n = 10
    x = pd.DataFrame(st.multivariate_normal(mean=[0, 0, 0], cov=1).rvs(size=n), 
                     columns=["x1", "x2", "x3"])
    x["id"] = np.arange(n)
    x["type"] = np.random.randint(5, size=n)
    x["size"] = np.random.randint(10, size=n)
    x["year"] = year
    x_list += [x]
x = pd.concat(x_list)

In [116]:
x.head()

,x1,x2,x3,id,type,size,year
0,-0.43060,-1.19393,-0.44430,0,2,9,2010
1,0.48941,-0.45156,0.58570,1,0,1,2010
2,1.17732,-0.96501,0.21828,2,3,6,2010
3,-0.86614,-0.32301,1.41292,3,3,5,2010
4,-0.71265,-1.36219,-1.70597,4,4,4,2010


In [115]:
px.scatter(x, x="x1", y="x2", 
           animation_frame="year", 
           animation_group="id",
           size="size", color="type", hover_name="size", 
           log_x = False, 
           size_max=45, 
           range_x=[-4,4], range_y=[-4,4],
           width=1000, height=1000,
           ) 